## Tensorflow Model Serving

### Using TF Serving

In [ ]:
# make and save model
from pathlib import Path
import tensorflow as tf

mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

tf.random.set_seed(42)
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

In [ ]:
# inspect SavedModel.
# Output will be a 'tag', which is a classification of metagraph(calculation graph + function signature(e.g. type, input & output size)).
!saved_model_cli show --dir '{model_path}'

In [ ]:
# Look at the 'tag set' above.
# Output will be a two signature definition, '__saved_model_init_op' and 'serving_default'.
!saved_model_cli show --dir '{model_path}' --tag_set serve

In [ ]:
# Look closely at the basic serving function 'serving_default'.
!saved_model_cli show --dir '{model_path}' --tag_set serve \
                      --signature_def serving_default

In [ ]:
# install tensorflow serving
url = "https://storage.googleapis.com/tensorflow-serving-apt"
src = "stable tensorflow-model-server tensorflow-model-server-universal"
!echo 'deb {url} {src}' > /etc/apt/sources.list.d/tensorflow-serving.list
!curl '{url}/tensorflow-serving.release.pub.gpg' | apt-key add -
!apt update -q && apt-get install -y tensorflow-model-server
%pip install -q -U tensorflow-serving-api==2.11.1

In [13]:
import os

os.environ["MODEL_DIR"] = str(model_path.parent.absolute())

In [14]:
# implement server
%%bash --bg
tensorflow_model_server \
    --port=8500 \
    --rest_api_port=8501 \
    --model_name=my_mnist_model \
    --model_base_path="${MODEL_DIR}" >my_server.log 2>&1

In [15]:
# query to TF serving using REST API
# make a request
import json

X_new = X_test[:3]
request_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist()
})

In [ ]:
# json is 100% text
request_json

In [ ]:
# deliver request data to TF serving using HTTP POST method
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
response = requests.post(server_url, data=request_json)
response.raise_for_status()
response = response.json()

In [ ]:
# make a prediction
import numpy as np
y_proba = np.array(response['predictions'])
y_proba.round(2)

In [ ]:
# query to TF serving using gRPC API
# Make a request.
# Make a PredictRequest protocol buffer and fill in fields.
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_sepc.signature_name = 'serving_default'
input_name = model.input_names[0]
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [ ]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

# make a channel
channel = grpc.insecure_channel('localhost:8500')

# make a gRPC service for the channel
predict_service = prediction_service_pb2_grpc.PredictServiceStub(channel)

# send a request
response = predict_service.Predict(request, timeout=10.0)

In [ ]:
# change protocol buffer to tensor
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

In [ ]:
# make a new version of model
np.random.seed(42)
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

In [ ]:
# save a new version of model
model_version = "0002"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")